# Step 1 - Read Data From Wikipedia

In [3]:
# main documentation page: http://beautiful-soup-4.readthedocs.io/en/latest/
# how to use the BeautifulSoup package: https://www.youtube.com/watch?v=ng2o98k983k video
from bs4 import BeautifulSoup 
import requests 
import pandas as pd

### Read Postcode Data from Wikipeadia


In [4]:
# read data

url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

source = requests.get(url).text
soup = BeautifulSoup(source,'lxml')

table = soup.find('table')
#table.prettify()

### Create Dataframe

In [5]:
# parse data and create dataframe 
postcode =[]
borough =[]
neighbourhood =[]

for tr in table.find_all('tr'):
    i = 1 
    for td in tr.find_all('td'):
        if i == 1:
            postcode.append(td.text)    
        elif i == 2:    
            borough.append(td.text)
        elif i == 3:        
            neighbourhood.append(td.text.replace('\n',''))
        i +=1
        
dict = {'PostalCode': postcode, 'Borough': borough, 'Neighbourhood': neighbourhood}        

df = pd.DataFrame.from_dict(dict)
df.head()

,PostalCode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


### Remove Borough = 'Not assigned'

In [6]:
# delete  rows with Borough = 'Not assigned'
df = df[df.Borough != 'Not assigned']
df.head()

,PostalCode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights


### Generate Groupby Postcode Result DataFrame

In [10]:
# group by Postcode and generate sets for Borough,Neighbourhood 
boroughLst = df.groupby(['PostalCode'])['Borough'].apply(set)
neighbourhoodLst = df.groupby(['PostalCode'])['Neighbourhood'].apply(set)

# create df_result by mergin 2 sets on join Postcode
df_result = pd.merge(pd.DataFrame(boroughLst), pd.DataFrame(neighbourhoodLst), how='outer', on=['PostalCode'])

# reset index to make Postcode columns 
df_result.reset_index(inplace=True)

# convert sets to String 
for i in range(len(df_result)-1):
    df_result.Neighbourhood[i] = ', '.join(df_result.Neighbourhood[i])
    df_result.Borough[i] = ', '.join(df_result.Borough[i])

# Update    Neighbourhood = 'Not assigned' with  Borough value. If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough. 
def myFun(Neighbourhood, Borough):
    if Neighbourhood =='Not assigned':
        return Borough
    else:
        return Neighbourhood
df_result.Neighbourhood = df_result.apply(lambda row :  myFun(row.Neighbourhood, row.Borough) , axis = 1) 

# show head 
df_result.head(12)


,PostalCode,Borough,Neighbourhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Port Union, Highland Creek, Rouge Hill"
2,M1E,Scarborough,"West Hill, Morningside, Guildwood"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park"
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge"
8,M1M,Scarborough,"Scarborough Village West, Cliffcrest, Cliffside"
9,M1N,Scarborough,"Cliffside West, Birch Cliff"


In [11]:
# M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma 
df_result[df_result.PostalCode =='M5A']

,PostalCode,Borough,Neighbourhood
53,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [12]:
# If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough. So for the 9th cell in the table on the Wikipedia page, the value of the Borough and the Neighborhood columns will be Queen's Park.
df_result[df_result.PostalCode =='M7A']

,PostalCode,Borough,Neighbourhood
85,M7A,Queen's Park,Queen's Park


### Print Shape Result

In [13]:
print('df_result.shape : ', df_result.shape)

df_result.shape :  (103, 3)


# Step 2 - Read Geocodes 

### Get geographical coordinates of each postal code

In [14]:
# read geocode coordinates from csv file
df_geocodes = pd.read_csv('http://cocl.us/Geospatial_data')
df_geocodes.rename(columns={"Postal Code": "PostalCode"}, inplace=True)
df_geocodes.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


### Merge df_geocodes with df_result 

In [15]:
#Merge df_geocodes with df_result 
df_result_geo = pd.merge(df_result, df_geocodes, how='outer', on=['PostalCode'])

# reset index to make PostalCode columns 
df_result_geo.reset_index(inplace=True)
df_result_geo = df_result_geo[['PostalCode','Borough','Neighbourhood', 'Latitude','Longitude']]
df_result_geo.head(12)


,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Port Union, Highland Creek, Rouge Hill",43.784535,-79.160497
2,M1E,Scarborough,"West Hill, Morningside, Guildwood",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",43.727929,-79.262029
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Scarborough Village West, Cliffcrest, Cliffside",43.716316,-79.239476
9,M1N,Scarborough,"Cliffside West, Birch Cliff",43.692657,-79.264848


# Step 3 - Visualize Map

### Generate Dataframe with Boroughs Contains Toronto

In [16]:
# generate df_toronto boroughs that contain the word Toronto
df_toronto = df_result_geo[df_result_geo.apply(lambda row : 'Toronto' in  row.Borough , axis = 1) ]
df_toronto.reset_index(inplace=True)
df_toronto = df_toronto[['PostalCode','Borough','Neighbourhood', 'Latitude','Longitude']]

print('df_toronto.shape : ', df_toronto.shape)
df_toronto.head()

df_toronto.shape :  (38, 5)


,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.676357,-79.293031
1,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
2,M4L,East Toronto,"India Bazaar, The Beaches West",43.668999,-79.315572
3,M4M,East Toronto,Studio District,43.659526,-79.340923
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790


### Create Map

In [18]:
#!conda install -c conda-forge folium=0.5.0
import folium # map rendering library
from geopy.geocoders import Nominatim

In [19]:
address = 'Toronto, Ontario, Canada'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


In [21]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=12)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_toronto['Latitude'], df_toronto['Longitude'], df_toronto['Borough'], df_toronto['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto